<テーブルスタイル= "幅：100％">
<tr>
<td style = "vertical-align：middle; text-align：left;">
<font size = "2">
<a href = "http://mng.bz/oryv"の補足コード>ゼロから大きな言語モデルを構築</a> <a href = "https://sebastianraschka.com"> sebastian raschka </a> <br>
<br>コードリポジトリ：<a href = "https://github.com/rasbt/llms-from-scratch"> https://github.com/rasbt/llms-from-scratch </a>
</font>
</td>
<td style = "vertical-align：middle; text-align：left;">
<a href = "http://mng.bz/oryv"> <img src = "https://sebastianraschka.com/images/llms-from-scratch-images/cover-small.webp" width = "100px"> </a>
</td>
</tr>
</table>

＃第2章：テキストデータの操作

このノートブックで使用されているパッケージ：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1
tiktoken version: 0.7.0


- この章では、データの準備とサンプリングをカバーして、LLMの入力データを「準備する」ことを取得します

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compression/01.webp?timestamp=1" width = "500px">

## 2.1単語の埋め込みを理解します

- このセクションにはコードがありません

- 埋め込みには多くの形態があります。この本のテキストの埋め込みに焦点を当てています

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/02.webp" width = "500px">

-LLMSは、高次元空間（つまり、数千の寸法）に埋め込みを使用して動作します
- このような高次元空間を視覚化できないため（私たち人間は1、2、または3つの寸法で考えています）、次の図は2次元の埋め込み空間を示しています

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/03.webp" width = "300px">

## 2.2トークン化テキスト

- このセクションでは、テキストをトークン化します。これは、個々の単語や句読点などの小さなユニットにテキストを壊すことを意味します

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/04.webp" width = "300px">

- 操作したい生のテキストをロードします
-  [エディス・ウォートンによる評決]（https://en.wikisource.org/wiki/the_verdict）はパブリックドメインの短編小説です

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- （以前のコードセルを実行するときに `ssl.sslcertverificationError`に遭遇した場合、それは古いPythonバージョンを使用することによる可能性があります。[https://github.com/rasbt/llms-from-scratch/pull/403）を見つけることができます））））））））））

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目標は、LLMのためにこのテキストをトークン化して埋め込むことです
- 上記のテキストに後で適用できるいくつかの簡単なサンプルテキストに基づいて、簡単なトークンザーを開発しましょう
- 次の正規表現は、空白に分割されます

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 私たちは、空白だけでなくコンマや期間に分割したいだけなので、それを行うために正規表現を変更しましょう

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 私たちが見ることができるように、これは空の文字列を作成します、それらを削除しましょう

In [6]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- これはかなり良さそうですが、期間、疑問符など、他の種類の句読点も処理しましょう

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- これはかなり良いです、そして私たちは今、このトークン化を生のテキストに適用する準備ができました

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/05.webp" width = "350px">

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- トークンの総数を計算しましょう

In [9]:
print(len(preprocessed))

4690


## 2.3トークンをトークンIDに変換します

- 次に、テキストトークンをトークンIDに変換し、後で埋め込むレイヤーを介して処理できます

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/06.webp" width = "500px">

- これらのトークンから、すべてのユニークなトークンで構成される語彙を構築できるようになりました

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下は、この語彙の最初の50エントリです。

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 以下では、小さな語彙を使用して短いサンプルテキストのトークン化を示します。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compression/07.webp?123" width = "500px">

- トークンザークラスにすべてをまとめる

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- 「エンコード」関数はテキストをトークンIDに変えます
- 「デコード」関数はトークンIDをテキストに戻します

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compression/08.webp?123" width = "500px">

- トークンザーを使用してテキストを整数にエンコードすることができます（つまり、トークン化）
- これらの整数は、LLMの入力として（後で）埋め込むことができます

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 整数をテキストにデコードすることができます

In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4特別なコンテキストトークンの追加

- 不明な単語のために「特別な」トークンを追加し、テキストの終わりを示すことは便利です

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compression/09.webp?123" width = "500px">

- 一部のトークンザーは、追加のコンテキストでLLMを支援するために特別なトークンを使用しています
- これらの特別なトークンの一部はそうです
-  `[bos]`（シーケンスの始まり）テキストの始まりをマークします
-  `[eos]`（シーケンスの終わり）テキストが終了するマーク（これは通常、複数の無関係なテキスト、例えば2つの異なるウィキペディア記事または2つの異なる本などを連結するために使用されます）
-  `[パッド]`（パディング）1を超えるバッチサイズのLLMSをトレーニングする場合（長さが異なる複数のテキストを含めることができます。パディングトークンを使用する
-  `[unk]`語彙に含まれていない単語を表すために

-GPT-2は上記のこれらのトークンを必要としないが、 `<| endoftext |>`トークンを使用して複雑さを減らすことに注意してください
-` <| endoftext |> `は上記の` [eos] `トークンに類似しています
-GPTはパディングに `<| endoftext |>`を使用します（通常、バッチ入力でトレーニングするときにマスクを使用するため、とにかくパッド入りトークンには出席しないので、これらのトークンが何であるかは関係ありません）
-GPT-2は、表記の単語に「 `トークンを使用しません。代わりに、GPT-2はバイトペアエンコード（BPE）トークンザーを使用します。これは、単語をサブワード単位に分解します。

-  `<| endoftext |>`トークンを使用します。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compression/10.webp" width = "500px">

- 次のテキストをトークン化するとどうなるか見てみましょう。

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上記は、語彙に「hello」という単語が含まれていないため、エラーを生成します
- そのような場合に対処するために、「 "" <| unk |> "`のような特別なトークンを語彙に追加して、未知の単語を表すことができます
- すでに語彙を拡張しているので、「 "<| endoftext |>" `という別のトークンを追加しましょう。これは、テキストの終わりを示すためにGPT-2トレーニングで使用されます（トレーニングデータセットが複数の記事、本などで構成されている場合など、連結テキスト間でも使用されます。

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- それに応じて、新しい ``トークンをいつ、どのように使用するかを知っているように、トークン剤を調整する必要があります

In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

修正されたトークン剤でテキストをトークン化してみましょう。

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5バイトペアエンコーディング

-GPT-2は、トークン剤としてbytepairエンコード（BPE）を使用しました
- モデルは、事前定義された語彙にない単語をより小さなサブワード単位または個々の文字に分解し、表記の単語を処理できるようにすることができます
- たとえば、GPT-2の語彙に「不慣れな言葉」という単語がない場合、訓練されたBPEの合併に応じて、["ungam"、 "airrial"、 "word"]またはその他のサブワードの内訳としてそれをトークン化するかもしれません。
- 元のBPEトークナイザーはこちらをご覧ください：[https://github.com/openai/gpt-2/blob/master/src/encoder.py]（https://github.com/openai/gpt-2/blob/master/src/encoder.py）
- この章では、Openaiのオープンソース[Tiktoken]（https://github.com/openai/tiktoken）ライブラリのBPEトークナイザーを使用しています。
-[./bytepair_encoder執事]にノートブックを作成しました（.../02_bonus_bytepair-encoder）これらの2つの実装を並べて比較しました（サンプルテキストではTiktokenは約5倍高速でした）

In [25]:
# pip install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


-BPEトークナーザーは、未知の単語をサブワードと個々の文字に分割します。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/11.webp" width = "300px">

## 2.6スライドウィンドウを使用したデータサンプリング

-LLMSを一度に1語ずつ生成するようにトレーニングするため、次の単語がシーケンスの次の単語が予測するターゲットを表している場合に、トレーニングデータを準備する必要があります。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/12.webp" width = "400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 各テキストチャンクについて、入力とターゲットが必要です
- モデルに次の単語を予測することを望むので、ターゲットは入力が右に1つの位置でシフトされたものです

In [31]:
enc_sample = enc_text[50:]

In [32]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


-  1つずつ、予測は次のようになります。

In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 注意メカニズムをカバーした後、後の章で次の単語の予測を処理します
- 今のところ、入力データセットを反復し、1つによってシフトされたターゲットを返すシンプルなデータローダーを実装します

-Pytorchのインストールとインポート（インストールのヒントについては、付録Aを参照）

In [35]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1


- スライドウィンドウアプローチを使用して、位置を+1で変更します。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/13.webp?123" width = "500px">

- 入力テキストデータセットからチャンクを抽出するデータセットとデータローダーを作成する

In [36]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- コンテキストサイズが4のLLMに対して、バッチサイズが1のバッチサイズでDataloaderをテストしましょう。

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 以下に示すように、コンテキストの長さ（こちら：4）に等しいストライドを使用する例：

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/14.webp" width = "500px">

- バッチ出力を作成することもできます
- ここでは、バッチ間に重複がないようにストライドを増やすことに注意してください。

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7トークンの埋め込みを作成します

- データはすでにLLMの準備ができています
- しかし最後に、埋め込み層を使用してトークンを連続ベクトル表現に埋め込みましょう
- 通常、これらの埋め込み層はLLM自体の一部であり、モデルトレーニング中に更新（トレーニング）されます

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/15.webp" width = "400px">

- 入力IDS 2、3、5、および1の次の4つの入力例があるとします（トークン化後）：

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])

- 簡単にするために、わずか6語の小さな語彙があり、サイズ3の埋め込みを作成したいとします。

In [43]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- これにより、6x3の重量マトリックスが発生します。

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


-  1ホットのエンコーディングに精通している人のために、上記のレイヤーアプローチは、本質的に1ホットエンコードを実装するより効率的な方法であり、その後に完全に接続されたレイヤーで行列をマルチプレイするためのより効率的な方法です。
- 埋め込み層は、1ホットのエンコーディングおよびマトリックスの拡大アプローチに相当するより効率的な実装であるため、バックプロパゲーションを介して最適化できるニューラルネットワーク層と見なすことができます。

-  ID 3でトークンを3次元ベクトルに変換するには、次のことを行います。

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 上記は `embedding_layer`重量マトリックスの4列目であることに注意してください
- 上記の4つの `input_ids`値すべてを埋め込むには、

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 埋め込み層は基本的にルックアップ操作です。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compression/16.webp?123" width = "500px">

-  **埋め込み層を通常の線形層と比較するボーナスコンテンツに興味があるかもしれません：[../03_bonus_embedding-vs-matmul]

## 2.8ワードポジションのエンコード

- 埋め込み層IDを入力シーケンスのどこにあるかに関係なく、IDを同一のベクトル表現に変換します。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compressed/17.webp" width = "400px">

- 位置埋め込みは、トークン埋め込みベクトルと組み合わされて、大規模な言語モデルの入力埋め込みを形成します。

<img src = "https://sebastianraschka.com/images/llms-from-scratch-images/ch02_compression/18.webp" width = "500px">

-Bytepairエンコーダーの語彙サイズは50,257です。
- 入力トークンを256次元ベクトル表現にエンコードするとします。

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

-DataloAderのデータをサンプリングすると、各バッチのトークンを256次元ベクトルに埋め込みました
- それぞれ4トークンの8つのバッチサイズがある場合、これにより8 x 4 x 256テンソルになります。

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


-GPT-2は絶対位置埋め込みを使用するため、別の埋め込み層を作成するだけです。

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


-  LLMで使用される入力埋め込みを作成するには、トークンと位置埋め込みを追加するだけです。

In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


- 入力処理ワークフローの初期段階では、入力テキストが個別のトークンにセグメント化されています
- このセグメンテーションに続いて、これらのトークンは、事前定義された語彙に基づいてトークンIDに変換されます。

<img src = "https://sebastianraschka.com/images/llms-from-from-scratch-images/ch02_compression/19.webp" width = "400px">

＃概要とテイクアウト

[./Dataloader.ipynb

エクササイズソリューションについては、[./exercise-solutions.ipynb]（./exercise-solutions.ipynb）を参照してください。

[バイトペアエンコーディング（BPE）トークナイザーをゼロから参照]